In [1]:
import torch

In [4]:
def synthetic(w,b,nums):
    if not isinstance(w,torch.Tensor):
        w = torch.tensor(w,dtype=torch.float32,requires_grad=True)
    
    x = torch.normal(0,1,(nums,len(w)))
    y = torch.matmul(x,w) + b
    y += torch.normal(0,1,y.shape)
    
    return x,y.reshape((-1,1))
    

In [8]:
x,y = synthetic([1,2],1,10)

In [63]:
def net(X,w,b):
    '''定义模型'''
    return torch.matmul(X,w) +b

def square_loss(y_hat,y):
    '''计算损失'''
    return (y-y_hat.reshape(y.shape))**2/2

def update_param(params,lr,bach_size):
    '''更新参数'''
    with torch.no_grad():
        for param in params:
            param -=lr*param.grad/bach_size
            param.grad.zero_()

In [58]:
import random
def data_iter(feature,y,batch_size):
    
    indices = list(range(len(feature)))
    random.shuffle(indices)
    for i in range(0,len(indices),batch_size):
        inde = indices[i:min(i+batch_size,len(indices)-1)]
        
        yield feature[inde],y[inde]
    

In [59]:
batch_size = 100
epochs = 5
lr = 1e-3
w,b = torch.normal(0,1,(2,1),requires_grad=True),torch.zeros(1,requires_grad=True)
feature,labels = synthetic([1,2],1,1000)

In [65]:
for epoch in range(epochs):
    for x,y in data_iter(feature,labels,batch_size):
        print(1)
        l = square_loss(net(x,w,b),y)
        l.sum().backward()
        update_param([w,b],lr,batch_size)
    with torch.no_grad():
        l = square_loss(net(feature,w,b),labels)
        l = loss.item()/len(feature)
        print(f"epoch={epoch+1},loss={l:0.3}")

1


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.